**NetCDF to xcube server example**

This example uses the raw .nc outputs from the DeepESDL polar science use case and converts them the xcube-ready data cubes.

This was originally run using a windows host machine whose D:\ drive was mapped to /mnt/d and where the netcdf files were saved under:
"D:\tmp\polarscience"


In [ ]:
import os
from rasterio.crs import CRS
import xarray as xr
from xcube.core.chunk import chunk_dataset
from xcube.core.store import new_data_store

root_input_dir = '/mnt/d/tmp/polarscience'

input_cases = {
    'amery_depth_temp': 'amery_damage_class_depth_temp.nc',
    'amery_surface_temp': 'amery_damage_class_surface_temp.nc', 
    'amery_uniform_temp': 'amery_damage_class_uniform_temp.nc'
}

chunk_size=256
crs_wkt = CRS.from_epsg(3031).to_wkt()

for scenario_name, file_name in input_cases.items():
    
    file_path = os.path.join(root_input_dir, file_name)

    dataset = xr.open_dataset(file_path)  # xarray handles the backend internally
    x = dataset["X_coord_(m)"][:, 0].values  # first row → x-coordinates
    y = dataset["Y_coord_(m)"][0, :].values  # first column → y-coordinates
    dataset = dataset.assign_coords(x=("x", x), y=("y", y))
    dataset = dataset.drop_vars(["X_coord_(m)", "Y_coord_(m)"])
    dataset["damage_class"] = dataset["damage_class"].rename({"r": "y", "c": "x"})  
    # Ensure data in y-first orientation!
    dataset["damage_class"] = dataset["damage_class"].transpose("y", "x")
    dataset.variables['damage_class'].attrs['4d_viewer_layer_type'] = 'heatmap'
    dataset['damage_class'].attrs['units'] = ' '
    dataset['damage_class'].attrs['long_name'] = scenario_name
    dataset['damage_class'].attrs['short_name'] = scenario_name    
    dataset['x'].attrs['axis'] = 'x'
    dataset['x'].attrs['standard_name'] = 'East'
    dataset['y'].attrs['axis'] = 'y'
    dataset['y'].attrs['standard_name'] = 'North'

    dataset.attrs['4d_viewer_ui_path'] = '/polar_science/'

    dataset['spatial_ref'] = xr.Variable((), 0)
    dataset['spatial_ref'].attrs.update({
        'grid_mapping_name': 'polar_stereographic',
        'crs_wkt': crs_wkt,
        'epsg_code': 'EPSG:3031',
        'spatial_ref': crs_wkt,
    })

    dataset['damage_class'].attrs['grid_mapping'] = 'spatial_ref'
  
    chunked_dataset = chunk_dataset(dataset, dict(x=chunk_size, y=chunk_size), format_name='zarr')

    store = new_data_store('file', root='./../xcube-server-data/')
    store.write_data(chunked_dataset, f'{scenario_name}.zarr', replace=True)
    store.write_data(chunked_dataset, f'{scenario_name}.levels', replace=True,
                     base_dataset_id=f'{scenario_name}.zarr', agg_methods="first")


/opt/conda/lib/python3.12/site-packages/xcube/util/plugin.py:176: UserWarning: Initializing xcube plugin 'xcube_4d_viewer' took 444 ms, consider code optimization. (For example, avoid eager import of packages, consider lazy loading of resources, etc.)
  warnings.warn(
